This is for merging all the truth tables together

In [1]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

#sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

import matplotlib.pyplot as plt 
import importlib
importlib.reload(VI)
pd.set_option('display.max_rows', 5)

In [2]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']

In [3]:
VI_dir = os.environ['HOME']+'/projects/VI_files/Andes_reinspection/ELG/'

In [4]:
vi = VI.read_individual_VI(VI_dir)

Reading all individual VI files in /global/homes/t/tlan/projects/VI_files/Andes_reinspection/ELG/
desi-vi_ELG_reinspection_67230_2_AJG.csv
desi-vi_ELG_reinspection_67230_4_CGS.csv
desi-vi_ELG_reinspection_67230_2_VRK.csv
desi-vi_ELG_reinspection_67230_7_CGS.csv
desi-vi_ELG_reinspection_67230_12_CGS.csv
desi-vi_ELG_reinspection_67230_12_HSH.csv
desi-vi_ELG_reinspection_67230_5_VRK.csv
desi-vi_ELG_reinspection_67230_12_CBA.csv
desi-vi_ELG_reinspection_67230_13_MAR.csv
desi-vi_ELG_reinspection_67230_13_CGS.csv
desi-vi_ELG_reinspection_67230_11_CGS.csv
desi-vi_ELG_reinspection_67230_11_CBA.csv
desi-vi_ELG_reinspection_67230_11_MAR.csv
desi-vi_ELG_reinspection_67230_1_VRK.csv
desi-vi_ELG_reinspection_67230_1_CGS.csv
desi-vi_ELG_reinspection_67230_5_CGS.csv
desi-vi_ELG_reinspection_67230_8_CGS.csv
desi-vi_ELG_reinspection_67230_14_MAR.csv
desi-vi_ELG_reinspection_67230_4_AJG.csv
desi-vi_ELG_reinspection_67230_9_CGS.csv
desi-vi_ELG_reinspection_67230_14_CGS.csv
desi-vi_ELG_reinspection_67230_

In [5]:
vi

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment
0,35185772160090388,55587,20200315,67230,0.34.4,-1,-1,GALAXY,0.7811,VRK,0,NaN,NaN,NaN,dubious fit (negative doublet and OIII); 2nd b...
1,35185772160091680,55587,20200315,67230,0.34.4,-1,-1,GALAXY,1.3426,VRK,4,NaN,NaN,NaN,resolved doublet; plausible Balmer region
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,35185766124488456,55587,20200315,67230,0.34.4,-1,-1,GALAXY,1.1028,CGS,3,NaN,NaN,NaN,NaN
1557,35185766124491776,55587,20200315,67230,0.34.4,-1,-1,GALAXY,0.5144,CGS,1,S,NaN,NaN,NaN


In [6]:
vi.loc[vi['VI_issue'].isnull(),'VI_issue']='--'
vi['all VI issue'] = vi.groupby('TARGETID')['VI_issue'].transform(lambda x: '|'.join(x))
unique_vi = vi.drop_duplicates(subset=['TARGETID'])

In [7]:
#vi[vi['TARGETID']==35191196661843077]
unique_vi
#vi.loc[vi['VI_comment'].isnull(),'VI_comment']='--'
#vi['all VI comments'] = vi.groupby('TARGETID')['VI_comment'].transform(lambda x: '|'.join(x))


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,all VI issue
0,35185772160090388,55587,20200315,67230,0.34.4,-1,-1,GALAXY,0.7811,VRK,0,--,NaN,NaN,dubious fit (negative doublet and OIII); 2nd b...,--|R
1,35185772160091680,55587,20200315,67230,0.34.4,-1,-1,GALAXY,1.3426,VRK,4,--,NaN,NaN,resolved doublet; plausible Balmer region,--|C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,35185766124491776,55587,20200315,67230,0.34.4,-1,-1,GALAXY,0.5144,VRK,1,--,NaN,NaN,doublet consistent with noise; fitted Balmer b...,--|S
1509,35185766120296280,55587,20200315,67230,0.34.4,-1,-1,GALAXY,1.4989,CGS,0,--,NaN,NaN,NaN,--


In [8]:
filenames = ['truth_table_Andes_reinspection_ELG_67230_20200315_1.csv', 
             'truth_table_Andes_reinspection_ELG_67230_20200315_2.csv', 
             'truth_table_Andes_reinspection_ELG_67230_20200315_3.csv',
             'truth_table_Andes_reinspection_ELG_67230_20200315_4.csv',
             'truth_table_Andes_reinspection_ELG_67230_20200315_5.csv']

truth_table = pd.read_csv(VI_dir+filenames[0])
for i in range(1,len(filenames)):
    partial_table = pd.read_csv(VI_dir+filenames[i])
    truth_table = truth_table.append(partial_table, ignore_index=True,sort=False)

In [9]:
truth_table=truth_table.merge(unique_vi[['TARGETID', 'all VI issue']], on='TARGETID', how='left')


In [10]:
truth_table

,TARGETID,Redrock_z,Redrock_spectype,best redshift,best spectype,vi_combined_flag,all VI comments,merger comment,N_VI,DELTACHI2,FIBER,all VI issue
0,35185735929695223,1.0936,GALAXY,1.0936,GALAXY,2.5,broad doublet + other (weak) lines|--,none,2,71.667133,328,--|--
1,35185735929696168,1.4989,GALAXY,1.4989,GALAXY,2.5,strong broad doublet + weak minor features may...,none,2,93.894897,301,--|--
...,...,...,...,...,...,...,...,...,...,...,...,...
678,35185802375857132,0.7850,GALAXY,0.7850,GALAXY,0.5,--|--,none,2,16.580985,3180,--|--
679,35185802375857926,1.4156,GALAXY,1.4156,GALAXY,2.0,--|--,disagreement in flag. Two emission line peaks ...,2,40.982959,3171,--|R


In [11]:
truth_table = truth_table.rename(columns={"best redshift":"best z", "vi_combined_flag":"best quality"})


In [12]:
truth_table.keys()

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best z', 'best spectype',
       'best quality', 'all VI comments', 'merger comment', 'N_VI',
       'DELTACHI2', 'FIBER', 'all VI issue'],
      dtype='object')

In [13]:
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')

for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

#EXPID = list(set(tf['EXPID']))[0]
#tf = tf[tf['EXPID']==EXPID]
tspec_df = tspec['TARGETID','ZWARN', 'ZERR'].to_pandas()
tf_df = tf['TARGETID','TARGET_RA','TARGET_DEC','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'].to_pandas()

unique_tf_df = tf_df.drop_duplicates(subset=['TARGETID'])
unique_tspec_df = tspec_df.drop_duplicates(subset=['TARGETID'])
#print(len(tspec_df))
#print(len(unique_tspec_df))
#print(len(tf_df))
#print(len(unique_tf_df))
#tf_df[tf_df['TARGETID']==35191200671599720]
truth_table = truth_table.merge(unique_tf_df, how='left', on='TARGETID')
truth_table = truth_table.merge(unique_tspec_df, how='left', on='TARGETID')
#tspec_df = tspec['TARGETID','ZWARN', 'ZERR' ].to_pandas()
#tf_df = tf['TARGETID','MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'].to_pandas()

#tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
#tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

#truth_table = truth_table.merge(tf_df, how='left', on='TargetID')
#truth_table = truth_table.merge(tspec_df, how='left', on='TARGETID')

In [14]:
truth_table.columns

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best z', 'best spectype',
       'best quality', 'all VI comments', 'merger comment', 'N_VI',
       'DELTACHI2', 'FIBER', 'all VI issue', 'TARGET_RA', 'TARGET_DEC',
       'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R',
       'FIBERFLUX_Z', 'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R',
       'MW_TRANSMISSION_Z', 'ZWARN', 'ZERR'],
      dtype='object')

In [15]:
len(truth_table)

680

In [16]:
truth_table = truth_table[['TARGETID','TARGET_RA','TARGET_DEC','Redrock_z', 'Redrock_spectype', 'best z', 'best spectype',
       'best quality', 'N_VI', 'all VI issue', 'all VI comments', 'merger comment', 'DELTACHI2', 'ZWARN', 'ZERR','FIBER', 
        'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'MW_TRANSMISSION_G',
       'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z']]

In [17]:
save_file = 'Truth_table_Andes_reinspection_ELG_67230_20200315_v1.csv'

In [18]:
truth_table.to_csv(VI_dir+save_file)
